In [4]:
# Hydra config
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# to remove prior config
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()  # Clear any previous initialization
# GlobalHydra.instance().initialize(config_path="config.yaml")
# IMPORTANT: the file path starts at the kernel level not the notebook
initialize(version_base=None, config_path="conf")
# cfg= compose(config_name="config")
cfg= compose(config_name="config", overrides=["learning=short","mlflow=different"])
# cfg= compose(config_name="config", overrides=["learning=long"])
print(OmegaConf.to_yaml(cfg))

learning_specs = cfg.learning
mlflow_specs=cfg.mlflow

learning_rate=learning_specs.learning_rate
batch_size=learning_specs.batch_size
num_epochs=learning_specs.num_epochs





db:
  driver: mysql
  user: root
  password: secret
  sausage: salami
learning:
  learning_rate: 0.001
  num_epochs: 1
  batch_size: 64
mlflow:
  uri: http://host.docker.internal:5000
  experiment_name: a more advanced experiment
  parameter: length



In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import mlflow
import mlflow.pytorch

# Set the MLflow tracking URI (use your own server or the default local URI)
mlflow.set_tracking_uri(mlflow_specs.uri)

# Create a MLflow experiment (you can change the experiment name)
mlflow.set_experiment(mlflow_specs.experiment_name)

# Create a PyTorch neural network model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = SimpleNet()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Load and preprocess the dataset (MNIST as an example)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
with mlflow.start_run() as run:
    mlflow.log_params({"learning_rate": learning_rate, "num_epochs": num_epochs, "batch_size": batch_size})

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Log metrics and save the model
        mlflow.log_metric("train_loss", loss.item(), step=epoch)
        mlflow.pytorch.log_model(model, "models")

print("\nexperiment run finished")

# You can view the results in the MLflow UI by running `mlflow ui` in your terminal


MlflowException: API request to http://host.docker.internal:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='host.docker.internal', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=a+more+advanced+experiment (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C10B779A10>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))

In [1]:
GlobalHydra.instance().clear()  # Clear any previous initialization
with initialize(config_path="notebooks/conf"):
    cfg = compose(config_name="config", overrides=["learning=short"])
    learning_specs = cfg.learning

    # Accessing specific values within the 'db' configuration
    learning_rate = learning_specs.learning_rate
    batch_size = learning_specs.batch_size
    num_epochs = learning_specs.num_epochs

    # Now you can use these values in your code

    # Create a logger for your experiment
    logger = hydra.utils.instantiate(cfg.logging)

    # Start logging the experiment information
    logger.info("Experiment started with the following parameters:")
    logger.info(OmegaConf.to_yaml(cfg))

    # Your training code here

    # Log important information during training
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Log loss for each batch
            logger.info(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(train_loader)}] Loss: {loss.item()}")

    # Log model and other relevant artifacts
    logger.info("Training completed.")
    logger.info("Logging model and artifacts.")
    # Log model, results, or other relevant artifacts using logger functions

    # End the experiment
    logger.info("Experiment completed.")


NameError: name 'GlobalHydra' is not defined